In [1]:
import requests
import pandas as pd
import numpy as np
import os
import time
import datetime
import json

In [2]:
csv_list = [file for file in os.listdir() if file.endswith("boxoffice.csv")]

In [3]:
for i, data in enumerate(csv_list):
    if i == 0:
        df = pd.read_csv(data)
    else:
        small_df = pd.read_csv(data)
        df = pd.concat([df, small_df]).reset_index(drop=True)

In [4]:
df.tail()

,순위,영화명,개봉일,매출액,매출액 점유율,관객수,스크린수,상영횟수,대표국적,국적,배급사
23429,2217,BISFF HONOR 2,NaN,0,0.0%,0,1,2,한국,한국,NaN
23430,2217,BISFF HOT HOT,NaN,0,0.0%,0,1,2,기타,기타,NaN
23431,2217,[인디포럼2016] 기획전03 김지현특별전 02,NaN,0,0.0%,0,1,1,한국,한국,NaN
23432,2217,영향 아래 있는 여자,NaN,0,0.0%,0,1,3,NaN,NaN,NaN
23433,합계,NaN,NaN,"820,067,267,748",NaN,"102,278,002","78,970","3,027,941",NaN,NaN,NaN


## 관객 수 10000명 이상

In [5]:
df["관객수"] = df["관객수"].str.replace(",","")

In [6]:
df["관객수"] = df["관객수"].astype(int)

In [7]:
movie_df = df[df["관객수"] > 10000].reset_index(drop=True)

In [8]:
movie_df = movie_df.drop(movie_df.index[[2569]])

In [9]:
movie_df = movie_df.drop_duplicates("영화명").reset_index(drop=True)

In [10]:
movieNm = movie_df["영화명"]

### movie_df to csv

In [11]:
movie_df.to_csv("movie.csv",encoding="utf-8")

## get_movie_data

In [3]:
boxoffice_df = pd.read_csv("boxoffice.csv")

In [4]:
movieNm = boxoffice_df["movieNm"]

In [6]:
def get_movie_data(movieNm):
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"
    params = {"key":"74ab5419e95d3e66cd760d6f498dc423", "movieNm":movieNm}
    r = requests.get(url, params=params)
    return r.json()

In [8]:
def make_movie_df(movieNm):
    movie_df = pd.DataFrame(columns = ["movieCd", "movieNm", "director","prdtYear", "openDt", "typeNm", "repNationNm", "repGenreNm", "companyNm"])
    for i in list(movieNm):
        try:
            for data in get_movie_data(i)['movieListResult']['movieList']:
                if len(data["directors"]) >= 2:
                    director = data["directors"][0]["peopleNm"]
                elif len(data["directors"]) == 1:
                    director = data["directors"][0]["peopleNm"]
                if len(data["companys"]) >= 2:
                    companyNm = data["companys"][0]["companyNm"]
                elif len(data["companys"]) == 1:
                    companyNm = data["companys"][0]["companyNm"] 
                movie_df.loc[len(movie_df)] = [
                    data["movieCd"],
                    data["movieNm"],
                    director,
                    data["prdtYear"],
                    data["openDt"],
                    data["typeNm"],
                    data["repNationNm"],
                    data["repGenreNm"],
                    companyNm
                ]
        except:
            print(i)
    return movie_df

In [9]:
movie_info_df = make_movie_df(movieNm)

팅커 테일러 솔저 스파이


In [11]:
len(movie_info_df)

6382

In [15]:
boxoffice_df

,Unnamed: 0,Unnamed: 0.1,movieCd,movieNm,director,openDt,typeNm,repNationNm,repGenreNm,showTm,watchGradeNm,actor_1,actor_2,actor_3,companyNm
0,0,0,20070316,디워,심형래,20070801,장편,한국,액션,90.0,12세이상관람가,브룩스 아만다,베어 제이슨,NaN,㈜영구아트
1,1,1,20122021,트랜스포머: 사라진 시대,마이클 베이,20140625,장편,미국,액션,165.0,12세이상관람가,마크 월버그,니콜라 펠츠,잭 레이너,씨제이이앤엠(주)
2,2,2,20110659,트랜스포머 3,마이클 베이,20110629,장편,미국,액션,152.0,12세이상관람가,조쉬 더하멜,존 말코비치,로지 헌팅턴-휘틀리,씨제이이앤엠(주)
3,3,3,20090303,트랜스포머: 패자의 역습,마이클 베이,20090624,장편,미국,SF,149.0,12세이상관람가,샤이아 라보프,메간 폭스,NaN,드림웍스 SKG
4,4,4,20070061,트랜스포머,마이클 베이,20070628,장편,미국,SF,135.0,12세이상관람가,샤이아 라보프,메간 폭스,NaN,드림웍스픽쳐스
5,5,5,20070209,화려한 휴가,김지훈,20070725,장편,한국,드라마,125.0,12세이상관람가,김상경,안성기,이요원,기획시대(주)
6,6,6,20070056,스파이더맨 3,샘 레이미,20070501,장편,미국,액션,139.0,12세이상관람가,토비 맥과이어,커스틴 던스트,NaN,콜럼비아 픽쳐스 코포레이션
7,7,7,20070057,캐리비안의 해적: 세상의 끝에서,고어 버빈스키,20070523,장편,미국,액션,168.0,12세이상관람가,조니 뎁,올랜도 블룸,NaN,제리 브룩하이머 필름
8,8,8,20070063,해리 포터와 불사조 기사단,데이빗 예이츠,20070711,장편,미국,판타지,137.0,전체관람가,다니엘 래드클리프,루퍼트 그린트,NaN,워너 브러더스 픽쳐스
9,9,9,20070060,다이 하드 4 : 죽어도 산다,렌 와이즈먼,20070717,장편,미국,액션,128.0,12세이상관람가,브루스 윌리스,매기 큐,NaN,샤이엔 엔터프라이지즈


In [21]:
movie_info_df= movie_info_df[["movieCd","prdtYear"]]

In [38]:
boxoffice_df["movieCd"] = boxoffice_df["movieCd"].apply(lambda x: str(x))

In [42]:
len(boxoffice_df)

2934

In [47]:
year_df

,Unnamed: 0,Unnamed: 0.1,movieCd,movieNm,director,openDt,typeNm,repNationNm,repGenreNm,showTm,watchGradeNm,actor_1,actor_2,actor_3,companyNm,prdtYear
0,0,0,20070316,디워,심형래,20070801,장편,한국,액션,90.0,12세이상관람가,브룩스 아만다,베어 제이슨,NaN,㈜영구아트,2007
1,1,1,20122021,트랜스포머: 사라진 시대,마이클 베이,20140625,장편,미국,액션,165.0,12세이상관람가,마크 월버그,니콜라 펠츠,잭 레이너,씨제이이앤엠(주),2014
2,1,1,20122021,트랜스포머: 사라진 시대,마이클 베이,20140625,장편,미국,액션,165.0,12세이상관람가,마크 월버그,니콜라 펠츠,잭 레이너,씨제이이앤엠(주),2014
3,2,2,20110659,트랜스포머 3,마이클 베이,20110629,장편,미국,액션,152.0,12세이상관람가,조쉬 더하멜,존 말코비치,로지 헌팅턴-휘틀리,씨제이이앤엠(주),2011
4,2,2,20110659,트랜스포머 3,마이클 베이,20110629,장편,미국,액션,152.0,12세이상관람가,조쉬 더하멜,존 말코비치,로지 헌팅턴-휘틀리,씨제이이앤엠(주),2011
5,3,3,20090303,트랜스포머: 패자의 역습,마이클 베이,20090624,장편,미국,SF,149.0,12세이상관람가,샤이아 라보프,메간 폭스,NaN,드림웍스 SKG,2009
6,3,3,20090303,트랜스포머: 패자의 역습,마이클 베이,20090624,장편,미국,SF,149.0,12세이상관람가,샤이아 라보프,메간 폭스,NaN,드림웍스 SKG,2009
7,4,4,20070061,트랜스포머,마이클 베이,20070628,장편,미국,SF,135.0,12세이상관람가,샤이아 라보프,메간 폭스,NaN,드림웍스픽쳐스,2007
8,5,5,20070209,화려한 휴가,김지훈,20070725,장편,한국,드라마,125.0,12세이상관람가,김상경,안성기,이요원,기획시대(주),2007
9,6,6,20070056,스파이더맨 3,샘 레이미,20070501,장편,미국,액션,139.0,12세이상관람가,토비 맥과이어,커스틴 던스트,NaN,콜럼비아 픽쳐스 코포레이션,


In [40]:
year_df = boxoffice_df.merge(movie_info_df,how="inner",on="movieCd")

In [54]:
year_df.to_csv("year.csv",encoding="utf-8")

In [16]:
movie_info_df = movie_info_df[~movie_info_df["movieNm"].str.contains("시네마정동")].reset_index(drop=True)

In [17]:
movie_info_df = movie_info_df[movie_info_df["repNationNm"] != "기타"].reset_index(drop=True)

In [18]:
movie_info_df = movie_info_df.drop_duplicates("movieNm").reset_index(drop=True)

In [19]:
movie_info_df.to_csv("movie_info.csv",encoding="utf-8")

In [48]:
movie_info_df.tail(20)

,Unnamed: 0,movieCd,movieNm,director,openDt,typeNm,repNationNm,repGenreNm,companyNm
3000,3000,20158905,버스 657,스콧 만,20160114,장편,미국,액션,미디어 아시아 필름즈
3001,3001,20040703,"조제, 호랑이 그리고 물고기들",이누도 잇신,20041029,장편,일본,드라마,아스믹에이스엔터테인먼트
3002,3002,20163081,미라클 프롬 헤븐,패트리시아 리건,20160525,장편,미국,드라마,아스믹에이스엔터테인먼트
3003,3003,20158982,트릭,이창열,20160713,장편,한국,스릴러,엘씨오픽쳐스
3004,3004,20154201,일렉트릭 슬라이드,트리스탄 패터슨,20150820,장편,미국,범죄,엘씨오픽쳐스
3005,3005,20164747,리틀 폭스,갈수영,20160623,장편,중국,애니메이션,엘씨오픽쳐스
3006,3006,19590006,벤허,윌리엄 와일러,19720907,장편,미국,드라마,메트로-골드윈-마이어
3007,3007,20152541,무수단,구모,20160303,장편,한국,미스터리,골든타이드픽처스(주)
3008,3008,20154902,자객 섭은낭,허우 샤오시엔,20160204,장편,대만,사극,골든타이드픽처스(주)
3009,3009,20156083,트윈스터즈,사만다 푸터먼,20160303,장편,미국,다큐멘터리,골든타이드픽처스(주)


In [58]:
def get_movie_detail(movieCd):
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json"
    params = {"key":"3805474b27f1763b1aaffa6702ea06e6", "movieCd":movieCd}
    r = requests.get(url, params=params)
    return r.json()

In [14]:
movie_info_df = pd.read_csv("movie_info.csv")

In [15]:
movieCd = movieCd.astype(str)

In [57]:
movieCd = movie_info_df["movieCd"]

In [44]:
def make_movie_detail_df(movie_info_df):
    movie_detail_df = pd.DataFrame(columns=["movieCd", 
                                            "movieNm", 
                                            "showTm", 
                                            "watchGradeNm", 
                                            "actor_1",
                                            "actor_2",
                                            "actor_3",
                                            "companyNm"
                                            ])
    for i in list(movieCd):
        try:
            data = get_movie_detail(i)['movieInfoResult']['movieInfo']
            actor_list = []
            if len(data["audits"]) >= 2:
                watchGradeNm = data["audits"][0]["watchGradeNm"]
            elif len(data["audits"]) == 1:
                watchGradeNm = data["audits"][0]["watchGradeNm"]
            if len(data["companys"]) >= 2:
                companyNm = data["companys"][0]["companyNm"]
            elif len(data["companys"]) == 1:
                companyNm = data["companys"][0]["companyNm"] 
            if len(data["actors"]) >= 3:
                actor_list = [
                    data["actors"][0]["peopleNm"],
                    data["actors"][1]["peopleNm"],
                    data["actors"][2]["peopleNm"]
                ]
            else:
                for i in range(len(data["actors"])):
                    actor_list.append(data["actors"][i]["peopleNm"])
                for i in range(3-len(data["actors"])):
                    actor_list.append("")
            movie_detail_df.loc[len(movie_detail_df)] = [
                data["movieCd"],
                data["movieNm"],
                data["showTm"],
                watchGradeNm,
                *actor_list,
                companyNm
            ]
        except:
             print(i)
    return movie_detail_df

In [59]:
movie_detail_df = make_movie_detail_df(movieCd)

20158905
20040703
20163081
20158982
20154201
20164747
19590006
20152541
20154902
20156083
20156080
20161598
20164387
20159965
20155174
20153182
20020227
20157441
20163441
20162101


In [52]:
movieCd = movie_info_df["movieCd"][3000::]

In [61]:
len(movie_detail_df)

3000

In [63]:
len(movie_detail1_df)

20

In [64]:
movie_detail = pd.concat([movie_detail_df, movie_detail1_df]).reset_index(drop=True)

In [66]:
movie_detail.to_csv("movie_detail.csv",encoding="utf-8")

In [86]:
boxoffice_df = movie_info_df.merge(movie_detail, left_on="movieNm", right_on="movieNm")[[
        "movieCd_x",
        "movieNm",
        "director",
        "openDt",
        "typeNm",
        "repNationNm",
        "repGenreNm",
        "showTm",
        "watchGradeNm",
        "actor_1",
        "actor_2",
        "actor_3",
        "companyNm_y"
    ]]

In [88]:
boxoffice_df = boxoffice_df.rename(columns={"movieCd_x":"movieCd", "companyNm_y":"companyNm"})

In [90]:
boxoffice_df.to_csv("boxoffice.csv",encoding="utf-8")